# Scraping www.timeanddate.com 

For Scraping web we should use these libs: 
- requests
- bs4

you should write your path in "path" of save and read files. Just remember that :D

In [ ]:
!pip3 install requests
!pip3 install bs4

In [ ]:
from bs4 import BeautifulSoup
import requests

This website has two parts:
- London Weather History for -- month year tabel
- it's javascript!

I scraped all data of each part and saved them in different CSV files.
let's start :)

In [ ]:
#get url
import urllib.parse as urlparse
url = "https://www.timeanddate.com/weather/uk/london/historic?month=&year="

# set months and years
month=range(6,13)
year=[2020,2021]

# add month and years to url
def insertChar(mystring, mont,year):
    mystring   =  mystring[:61] + mont + mystring[61:67]+ year + mystring[67:]
    return mystring   

result_url=[]
for i in month:
  for j in year:    
    result_url.append(insertChar(url, str(i),str(j)))

# print all urls
result_url

['https://www.timeanddate.com/weather/uk/london/historic?month=6&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=6&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=7&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=7&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=8&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=8&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=9&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=9&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=10&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=10&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=11&year=2020',
 'https://www.timeanddate.com/weather/uk/london/historic?month=11&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=12&year=2

I got all contents of these urls and save them into the text file with the below code. Now we have all thing that we wanted!


In [ ]:
# get content of web for each date 
for idx,url in enumerate(result_url):
  months_weather = requests.get(str(url))
  soup=BeautifulSoup(months_weather.content, 'html.parser')
  #print(months_weather)

  #save them into the text file
  with open("/path/{}soupFile.txt".format(idx), "w") as f:
        f.write(str(soup))
  

In [ ]:
# for read soupfiles- it's just a test :)
for idx in range(0,13):
  f = open("/path/{}soupFile.txt".format(idx))
  soup = f.read()
  bs = BeautifulSoup(soup,"html.parser")

The script part has data of all days of months that I set them in the first part.
In the first part, I set months: (6-12) and now I want to catch weather's of 30 days of all these months in 2 years(2020 and 2021). 


In [ ]:
#script part
import json
dfs = pd.DataFrame()
for idx in range(0,13):
  ddf = pd.DataFrame(columns=['date', 'date_id','wind', 'desc','temp','baro','hum'])
  f = open("/path/{}soupFile.txt".format(idx))
  r = f.read()
  soup = BeautifulSoup(r,"html.parser")
  scripts = soup.find_all('script')
  jsonObj = None
  for script in scripts:
      if 'var data='  in script.text:
          jsonStr = script.text.strip()

          jsonStr = jsonStr.split('var data=')[1]
          jsonStr = jsonStr.split(';')[0]

          jsonObj = json.loads(jsonStr)
  for item in jsonObj['detail']:
    date = item['ds']
    date_id=item['date']
    wind = item['wind']
    desc=item['desc']
    temp=item['temp']
    baro=item['baro']
    hum=item['hum']
    ddf = ddf.append({'date': date, 'date_id': date_id, 'wind':wind, 'desc': desc, 'temp': temp, 'baro': baro, 'hum': hum }, ignore_index=True)
  dfs = pd.concat([dfs, ddf], axis=0).reset_index(drop=True)

# print 
dfs.head(5)


The table part has 24h on the first day of each month and I catch its data in two ways that we can see in continue :)

In [ ]:
# London Weather History for --month year table part
# way with tags 
dft = pd.DataFrame()
for idx in range(0,13):
  f = open("/path/{}soupFile.txt".format(idx))
  r = f.read()
  soup = BeautifulSoup(r,"html.parser")
  table = soup.find('table', id='wt-his')
  df = pd.DataFrame(columns=['time', 'weather','Humidity', 'wind','temp'])
  # Collecting Ddata
  for row in table.tbody.find_all('tr'):   
      # Find all data for each column
      time=[sd.get_text() for sd in row.select("th")][0]
      weather=[t.get_text() for t in row.find_all('td', class_="small")][0]
      wind =[w.get_text() for w in row.find_all('td', class_="sep")][0]
      Humidity =[wh.get_text() for wh in row.find_all('td', class_="sep")][1]
      temp = [tp.get_text() for tp in row.find_all('td', class_="")][0]
      
      df = df.append({'time': time, 'weather': weather,'Humidity':Humidity, 'wind': wind, 'temp': temp }, ignore_index=True)
  dft = pd.concat([dft, df], axis=0).reset_index(drop=True)

dft.head(5)

,time,weather,Humidity,wind,temp
0,"00:50Mon, 1 Jun",Clear.,1023 mbar,15 km/h,14 °C
1,01:50,Clear.,1023 mbar,13 km/h,13 °C
2,02:50,Clear.,1023 mbar,11 km/h,13 °C
3,03:50,Clear.,1023 mbar,9 km/h,12 °C
4,04:50,Sunny.,1023 mbar,9 km/h,11 °C
...,...,...,...,...,...
422,19:50,Passing clouds.,1026 mbar,7 km/h,2 °C
423,20:50,Clear.,1026 mbar,4 km/h,3 °C
424,21:50,Partly cloudy.,1026 mbar,7 km/h,3 °C
425,22:50,Passing clouds.,1025 mbar,7 km/h,2 °C


In [ ]:
# way with pd.read_html
for idx in range(0,13):
  f = open("/path/{}soupFile.txt".format(idx))
  r = f.read()
  soup = BeautifulSoup(r,"html.parser")
  dfss = pd.DataFrame()
  table = soup.find('table', id='wt-his')
  dff = pd.read_html(str(table))[0]
  # Dropping a level down
  dff.columns = dff.columns.droplevel(0)
  dff=dff.drop(['Unnamed: 1_level_1','Unnamed: 5_level_1','Visibility'], axis = 1)
  dfss = pd.concat([dfss, dff], axis=0).reset_index(drop=True)

dfss.head(5)

,Time,Temp,Weather,Wind,Humidity,Barometer
0,"00:50Tue, 1 Dec",5 °C,Clear.,17 km/h,81%,1023 mbar
1,01:50,5 °C,Clear.,17 km/h,76%,1023 mbar
2,02:50,4 °C,Clear.,13 km/h,81%,1023 mbar
3,03:50,4 °C,Clear.,13 km/h,81%,1024 mbar
4,04:50,4 °C,Clear.,13 km/h,81%,1024 mbar
5,05:50,3 °C,Clear.,11 km/h,87%,1024 mbar
6,06:50,2 °C,Clear.,7 km/h,93%,1025 mbar
7,07:50,2 °C,Passing clouds.,9 km/h,93%,1025 mbar
8,08:50,3 °C,Sunny.,11 km/h,87%,1026 mbar
9,09:50,5 °C,Sunny.,15 km/h,81%,1026 mbar


In [ ]:
#save all dataframes in CSV files
dfss.to_csv('/path/Weather_london_2.csv', sep=',')
dfs.to_csv('/path/Weather_london_1.csv', sep=',')
dft.to_csv('/path/Weather_london_3.csv', sep=',')